# Trials Generation Script

## Loads all stimuli details from a CSV and generates trial sets

### Key features of each stimulus are also included in dataframe rows (CSV rows). 
 
Raúl Arrabales - Change Blindness in Hierarchical Stimuli Project

In [5]:
from os import listdir
from os.path import isfile, join

import pandas as pd

In [6]:
# Load dataframe with all stimuli
sdf = pd.read_csv('FilteredCircleStimuli.csv',sep=';')

In [7]:
# Stimuli table
sdf.head(10)

,StimulusId,FileName,Name,Pattern,G1,L1
0,0,Za.bmp,Za,Circle,Z,A
1,1,Zc.bmp,Zc,Circle,Z,C
2,2,Ze.bmp,Ze,Circle,Z,E
3,3,Zf.bmp,Zf,Circle,Z,F
4,4,Zh.bmp,Zh,Circle,Z,H
5,5,Zk.bmp,Zk,Circle,Z,K
6,6,Zm.bmp,Zm,Circle,Z,M
7,7,Zn.bmp,Zn,Circle,Z,N
8,8,Zo.bmp,Zo,Circle,Z,O
9,9,Zu.bmp,Zu,Circle,Z,U


In [8]:
# Stimuli table stats
sdf.describe()

,StimulusId
count,110.000000
mean,54.500000
std,31.898276
min,0.000000
25%,27.250000
50%,54.500000
75%,81.750000
max,109.000000


In [64]:
# Dataframe to store all trials
trial_columns = [
    'TrialId',     # Trial ID
    's1',          # Stimulus 1
    's2',          # Stimulus 2
    's3',   
    's4',   
    's5',   
    's6',   
    's7',   
    's8',   
    's9',   
    'x9',         # Position of changed stimulus
    'y9',
    'changecorrect', # Position num (1..8) corresponding to x9,y9
    'changetype',  # L, G, LG
    'firstG',  # First Global letter
    'secondG', # Second Global letter
    'fisrtL',  # First Local letter
    'secondL'] # Second Local letter
trials_df = pd.DataFrame(columns=trial_columns)
trials_df.head()

,TrialId,s1,s2,s3,s4,s5,s6,s7,s8,s9,x9,y9,changecorrect,changetype,firstG,secondG,fisrtL,secondL


In [40]:
# Dataframe with positions
pos_columns = ['x','y','positionnum']
pos_df = pd.DataFrame(columns=pos_columns)
pos_df.loc[len(pos_df)] = ['50','15','1']
pos_df.loc[len(pos_df)] = ['71','28','2']
pos_df.loc[len(pos_df)] = ['80','50','3']
pos_df.loc[len(pos_df)] = ['71','73','4']
pos_df.loc[len(pos_df)] = ['50','85','5']
pos_df.loc[len(pos_df)] = ['29','73','6']
pos_df.loc[len(pos_df)] = ['20','50','7']
pos_df.loc[len(pos_df)] = ['29','27','8']
pos_df.head(10)

,x,y,positionnum
0,50,15,1
1,71,28,2
2,80,50,3
3,71,73,4
4,50,85,5
5,29,73,6
6,20,50,7
7,29,27,8


In [122]:
# Iterate over all rows combined with all (same) rows - NxN combinations

trials_df = pd.DataFrame(columns=trial_columns) # Reset trials df
i = 0      # trial id
num_G = 0
num_L = 0
num_GL = 0
added_G = 0
added_L = 0
added_GL = 0
num_Trials = 1000

print('Processing... Generating ' + str(num_Trials) + ' trials')


while (i < num_Trials):
    
    
    # Get one random row (trial)
    GCrow = ['s1','s2','s3','s4','s5','s6','s7','s8','s9','x9','y9','pos']
    # print(GCrow)
    
    # Random fill of the row
    randomStimuli = sdf.sample(n=9)
    # print(randomStimuli)
    for j in range(0,9):
        GCrow[j] = randomStimuli.iloc[j]['FileName']    
        
    # Get a random position
    randomPos = pos_df.sample(n=1)
    GCrow[9] = randomPos.loc[:,'x'].tolist()[0]
    GCrow[10] = randomPos.loc[:,'y'].tolist()[0]
    GCrow[11] = randomPos.loc[:,'positionnum'].tolist()[0]
    
    # Determine type of change:
    L1 = randomStimuli.iloc[int(GCrow[11])-1]['L1']
    L2 = randomStimuli.iloc[8]['L1']
    G1 = randomStimuli.iloc[int(GCrow[11])-1]['G1']
    G2 = randomStimuli.iloc[8]['G1']    
    # print(G1+'->'+G2+' '+L1+'->'+L2)            
    # print(GCrow)
    
    ChangeType = 'N/A'
    if (L1 == L2 and G1 == G2):
        ChangeType = "N"
    elif (L1 == L2):
        ChangeType = 'G'
        num_G += 1
    elif (G1 == G2):
        ChangeType = 'L'
        num_L += 1
    else:
        ChangeType = 'GL'
        num_GL += 1
    
    # print('ChangeType: ' + ChangeType + ' ' +
    #   str(i) + ' trials. (' + 
    #   str(num_G)+' Global changes, '+ 
    #   str(num_L)+' Local changes, '+
    #   str(num_GL)+' Global and Local changes)')

    # if the trial is good, add it to the trial set: 
    if ( (ChangeType == 'G' and num_G < (num_Trials/3)+1) or 
         (ChangeType == 'L' and num_L < (num_Trials/3)+1) or 
         (ChangeType == 'GL' and num_GL < (num_Trials/3)+1) ):
           
    
        # Update stimulus index
        i += 1
    
        # Add row to trial df
        trials_df.loc[len(trials_df)] = [
            str(i),                # Trial ID
            GCrow[0], 
            GCrow[1], 
            GCrow[2], 
            GCrow[3], 
            GCrow[4], 
            GCrow[5], 
            GCrow[6], 
            GCrow[7], 
            GCrow[8], 
            GCrow[9], 
            GCrow[10], 
            GCrow[11], 
            ChangeType, 
            G1, 
            G2, 
            L1,
            L2]
        
        if (ChangeType == 'G'):
            added_G += 1
        elif (ChangeType == 'L'):
            added_L += 1
        elif (ChangeType == 'GL'):
            added_GL += 1
        else:
            print("Change type Error!!!")
    
print('Done. Processed: ' +
      str(i) + ' trials. (' + 
      str(added_G)+' Global changes, '+ 
      str(added_L)+' Local changes, '+
      str(added_GL)+' Global and Local changes)')


Processing... Generating 1000 trials
Done. Processed: 1000 trials. (334 Global changes, 332 Local changes, 334 Global and Local changes)


In [123]:
trials_df.head(10)                         

,TrialId,s1,s2,s3,s4,s5,s6,s7,s8,s9,x9,y9,changecorrect,changetype,firstG,secondG,fisrtL,secondL
0,1,Fa.bmp,Ha.bmp,Ne.bmp,Fh.bmp,Ak.bmp,Fc.bmp,Uz.bmp,Fn.bmp,Nc.bmp,20,50,7,GL,U,N,Z,C
1,2,Zc.bmp,Me.bmp,Ek.bmp,Au.bmp,Mn.bmp,Um.bmp,Ea.bmp,Eo.bmp,Fe.bmp,50,15,1,GL,Z,F,C,E
2,3,Oa.bmp,Fa.bmp,Hc.bmp,Fk.bmp,Mk.bmp,Fh.bmp,Uc.bmp,Ze.bmp,On.bmp,50,15,1,L,O,O,A,N
3,4,Ko.bmp,Af.bmp,Uh.bmp,Uk.bmp,Om.bmp,Ce.bmp,Nc.bmp,Oe.bmp,Za.bmp,50,85,5,GL,O,Z,M,A
4,5,Za.bmp,Fk.bmp,Nh.bmp,Ao.bmp,Uk.bmp,Ef.bmp,Cn.bmp,Ka.bmp,Kc.bmp,50,15,1,GL,Z,K,A,C
5,6,Uz.bmp,Fm.bmp,Cz.bmp,Mn.bmp,Kn.bmp,Ek.bmp,Ke.bmp,Mz.bmp,Cf.bmp,71,73,4,GL,M,C,N,F
6,7,Ma.bmp,Cz.bmp,Ez.bmp,Kh.bmp,Oe.bmp,Ha.bmp,Zu.bmp,Zc.bmp,Me.bmp,29,27,8,GL,Z,M,C,E
7,8,Zc.bmp,Kn.bmp,Ca.bmp,Nk.bmp,En.bmp,Uk.bmp,Co.bmp,Zk.bmp,Ck.bmp,29,27,8,G,Z,C,K,K
8,9,Ef.bmp,Ou.bmp,Zn.bmp,Hz.bmp,Uk.bmp,Hm.bmp,Ck.bmp,Eo.bmp,Kn.bmp,50,15,1,GL,E,K,F,N
9,10,Zo.bmp,Uk.bmp,Fe.bmp,Of.bmp,Cz.bmp,Fn.bmp,Eh.bmp,Ck.bmp,Fu.bmp,80,50,3,L,F,F,E,U


In [124]:
print('Total number of trials generated: ' + str(len(trials_df)))

Total number of trials generated: 1000


In [119]:
# Trials Dataframe samlple: 
trials_df.head(10)

,TrialId,s1,s2,s3,s4,s5,s6,s7,s8,s9,x9,y9,changecorrect,changetype,firstG,secondG,fisrtL,secondL
0,1,Kc.bmp,Eu.bmp,Of.bmp,Me.bmp,Zf.bmp,Um.bmp,Fa.bmp,Ko.bmp,Fz.bmp,71,28,2,GL,E,F,U,Z
1,2,Hm.bmp,Kn.bmp,An.bmp,Nc.bmp,Zc.bmp,Fe.bmp,Hn.bmp,Zk.bmp,Fk.bmp,29,27,8,G,Z,F,K,K
2,3,Uh.bmp,Zu.bmp,Kh.bmp,Ck.bmp,Zm.bmp,Nh.bmp,Ne.bmp,Em.bmp,Ec.bmp,71,73,4,GL,C,E,K,C
3,4,Ec.bmp,Ze.bmp,Ao.bmp,Mo.bmp,Hf.bmp,Ua.bmp,Eu.bmp,Ou.bmp,Ce.bmp,50,15,1,GL,E,C,C,E
4,5,Cu.bmp,Mn.bmp,Mh.bmp,Zk.bmp,Nf.bmp,Cz.bmp,Mo.bmp,Cm.bmp,He.bmp,29,27,8,GL,C,H,M,E
5,6,Oe.bmp,Ac.bmp,Ef.bmp,Uc.bmp,Af.bmp,Om.bmp,Mz.bmp,Nc.bmp,Ze.bmp,50,15,1,G,O,Z,E,E
6,7,Nk.bmp,Ce.bmp,Hk.bmp,Fz.bmp,Ch.bmp,Um.bmp,Uc.bmp,Ze.bmp,Co.bmp,29,27,8,GL,Z,C,E,O
7,8,Fk.bmp,Ho.bmp,Hz.bmp,Co.bmp,Om.bmp,Na.bmp,Mz.bmp,Az.bmp,Oa.bmp,20,50,7,GL,M,O,Z,A
8,9,Ca.bmp,Ku.bmp,Ec.bmp,Ke.bmp,Zk.bmp,Ok.bmp,Uc.bmp,Ne.bmp,Nf.bmp,50,15,1,GL,C,N,A,F
9,10,He.bmp,Fh.bmp,Fc.bmp,Uf.bmp,Ka.bmp,Hk.bmp,Hm.bmp,Em.bmp,Za.bmp,29,73,6,GL,H,Z,K,A


In [125]:
# Export trials dataframe to a CSV file:
trials_df.to_csv('AllCircleTrials.csv', sep =';', index_label='Id')